In [ ]:
!pip install -r requirements.txt

In [ ]:
import os
import json
import cv2 as cv
import pandas as pd
import plotly.express as px
from sklearn.feature_selection import mutual_info_regression

In [ ]:
!unzip -o maskrcnn/output/test/dt_masks.zip

In [5]:
%run scripts/all_measures_to_json.py
%run scripts/merge_anns_and_measures.py
%run scripts/evaluate_network_performance.py
%run scripts/evaluate_measuring_performance.py
%run scripts/compute_geometrical_features.py

In [ ]:
df = pd.read_csv("data/geometric_features.csv")
df_ = df[df["detected_mask_id"] >= 0]

## Mask R-CNN evaluation

In [ ]:
total_loss = pd.read_csv("maskrcnn/eval metrics/total_loss.csv")

total_loss.drop("Wall time", axis=1, inplace=True)
total_loss.columns = ["Iteration", "Loss"]

fig = px.line(total_loss, x="Iteration", y="Loss", height=300, width=500)
fig.update_layout(font_size=15)
fig.write_image("data/total_loss.png")
fig.show()

In [ ]:
with open("maskrcnn/eval metrics/time_seconds.json") as f:
    other_data = json.load(f)

pd.DataFrame([other_data.values()], columns=other_data.keys())

In [ ]:
df_summary = pd.read_csv("data/data_summary.csv")
df_summary

In [ ]:
fig = px.histogram(df, x="iou", height=300, width=500)
fig.update_layout(font_size=15, xaxis_title="IoU")
fig.write_image("data/ious_histogram.png")
fig.show()

In [ ]:
features = ["iou", "area", "perimeter", "circularity", "aspect_ratio", "rectangularity", "solidity"]
features_= ["IoU", "Area", "Perimeter", "Circularity", "Aspect Ratio", "Rectangularity", "Solidity"]

all_ft_mi = []
for x,y in zip(features, features_):
    result = mutual_info_regression(df[features], df[x], discrete_features=[False for _ in features])
    all_ft_mi.append(pd.DataFrame(result.T, columns=[y], index=features_))

ft_mi = pd.concat(all_ft_mi, axis=1)
ft_mi = ft_mi.applymap(lambda x: "{:.2f}".format(x))

px.imshow(ft_mi, text_auto=True).update_layout(font_size=16).write_image("data/features_iou_heatmap.png")

## Measurement evaluation

In [ ]:
algorithms = ["pca", "lr", "centroid", "brute-force"]

errors = pd.DataFrame([], columns=algorithms)
times  = pd.DataFrame([], columns=algorithms)

for alg in algorithms:
    errors[alg] = abs(df_["ground-truth_measure"] - df_[f"{alg}_measure"]) / df_["ground-truth_measure"]
    times[alg] = df_[f"{alg}_time"]

times *= 1000

In [ ]:
fig = px.box(errors,
    color_discrete_sequence=["coral"],
    height=500, width=500,
    labels= {"value": "Error"}
).update_layout(font_size=15, xaxis_title=None)
fig.show()
fig.write_image("data/measuring_boxplot.png")

In [ ]:
summary = pd.concat([errors.mean()*100, errors.std()*100, times.mean(), times.std()], axis=1)
summary.columns = ["average error", "error STD", "average time (ms)", "time STD (ms)"]

for k in summary.columns:
    if "error" in k:
        summary[k] = summary[k].apply(lambda x: "{:.1f}%".format(x)) 
    else: summary[k] = summary[k].apply(lambda x: "{:.1f}".format(x))

summary